In [1]:
import os
import re
import random
import hashlib

from enum import Enum
from collections import ChainMap

import numpy as np

from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

from scipy.io import wavfile

import librosa

import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline

In [2]:
#########
# Utils #
#########

class SimpleCache(dict):
    def __init__(self, limit):
        super().__init__()
        self.limit = limit
        self.n_keys = 0

    def __setitem__(self, key, value):
        if key in self.keys():
            super().__setitem__(key, value)
        elif self.n_keys < self.limit:
            self.n_keys += 1
            super().__setitem__(key, value)
        return value

class SerializableModule(nn.Module):
    def __init__(self):
        super().__init__()

    def save(self, filename):
        torch.save(self.state_dict(), filename)

    def load(self, filename):
        self.load_state_dict(torch.load(filename, map_location=lambda storage, loc: storage))

def play_wav(wav_path):
    sample_rate, samples = wavfile.read(wav_path)
    return ipd.Audio(samples, rate=sample_rate)

In [3]:
class DatasetType(Enum):
    TRAIN = 0
    DEV = 1
    TEST = 2

In [8]:
# useful PyTorch abstraction
class SpeechDataset(data.Dataset):
    def __init__(self, data, set_type, config):
        ####
        # data – {wave_file_path : label}
        # set_type – DatasetType (dev, train or test)
        # config.bg_noise_files – will be used to noise wavs during training (see preprocess func)
        ####
        super().__init__()
        self.audio_files = list(data.keys())
        self.audio_labels = list(data.values())
        self.set_type = set_type
        config["bg_noise_files"] = list(filter(lambda x: x.endswith("wav"), config.get("bg_noise_files", [])))
        self.bg_noise_audio = [librosa.core.load(file, sr=16000)[0] for file in config["bg_noise_files"]]
        self.unknown_prob = config["unknown_prob"]
        self.silence_prob = config["silence_prob"]
        self.noise_prob = config["noise_prob"]
        self.n_dct = config["n_dct_filters"]
        self.n_mels = config["n_mels"]
        self.input_length = config["input_length"]
        self.timeshift_ms = config["timeshift_ms"]
        self.filters = librosa.filters.dct(config["n_dct_filters"], config["n_mels"])
        self._audio_cache = SimpleCache(config["cache_size"])
        self._file_cache = SimpleCache(config["cache_size"])
        n_unk = len(list(filter(lambda x: x == 1, self.audio_labels)))
        self.n_silence = int(self.silence_prob * (len(self.audio_labels) - n_unk))
    
    # PyTorch needs this function
    def __getitem__(self, index):
        if index >= len(self.audio_labels):
            return self.preprocess(None, silence=True), 0
        return self.preprocess(self.audio_files[index]), self.audio_labels[index]

    # PyTorch needs this function
    def __len__(self):
        return len(self.audio_labels) + self.n_silence

    # Our extra functions go below:
    def _timeshift_audio(self, data):
        shift = (16000 * self.timeshift_ms) // 1000
        shift = random.randint(-shift, shift)
        a = -min(0, shift)
        b = max(0, shift)
        data = np.pad(data, (a, b), "constant")
        return data[:len(data) - a] if a else data[b:]
    
    def _preprocess_audio(self, data, n_mels, dct_filters):
        data = librosa.feature.melspectrogram(
            data, 
            sr=16000, 
            n_mels=n_mels, 
            hop_length=160, 
            n_fft=480, 
            fmin=20, 
            fmax=4000
        )
        data[data > 0] = np.log(data[data > 0])
        data = [np.matmul(dct_filters, x) for x in np.split(data, data.shape[1], axis=1)]
        data = np.array(data, order="F").squeeze(2).astype(np.float32)
        return data

    def preprocess(self, example, silence=False):
        if silence:
            example = "__silence__"
        if random.random() < 0.7:
            try:
                return self._audio_cache[example]
            except KeyError:
                pass
        
        in_len = self.input_length
        if self.bg_noise_audio:
            # take a random samples chunk of a random noise audio
            bg_noise = random.choice(self.bg_noise_audio)
            a = random.randint(0, len(bg_noise) - in_len - 1)
            bg_noise = bg_noise[a:a + in_len]
        else:
            bg_noise = np.zeros(in_len)
        
        use_clean = (self.set_type != DatasetType.TRAIN)
        if use_clean:
            bg_noise = np.zeros(in_len)
        if silence:
            data = np.zeros(in_len, dtype=np.float32)
        else:
            file_data = self._file_cache.get(example)
            data = librosa.core.load(example, sr=16000)[0] if file_data is None else file_data
            self._file_cache[example] = data
        data = np.pad(data, (0, max(0, in_len - len(data))), "constant")
        if not use_clean:
            data = self._timeshift_audio(data)

        if random.random() < self.noise_prob or silence:
            a = random.random() * 0.1
            data = np.clip(a * bg_noise + data, -1, 1)
        data = torch.from_numpy(self._preprocess_audio(data, self.n_mels, self.filters))
        self._audio_cache[example] = data
        return data

### Define config

In [4]:
config = dict(
    data_folder="./../raw_data/train/audio",
    wanted_words="yes no up down left right on off stop go".split(),
    
    seed=0,
    
    output_file="./../models/model.pt", 
    gpu_no=0,
    cache_size=32768,
    
    n_epochs=26,
    momentum=0.9, 
    weight_decay=0.00001,
    lr=[0.1, 0.01, 0.001], 
    schedule=[3000, 6000], 
    batch_size=64, 
    dev_every=1,
    use_nesterov=False,
    
    group_speakers_by_id=True,
    
    silence_prob=0.1,
    noise_prob=0.8,
    unknown_prob=0.1, # percentage of unknown waves in the train/dev/test dataset
    
    input_length=16000,
    timeshift_ms=100,
    # Below are two parameters of Discrete Cosine Transform (http://en.wikipedia.org/wiki/Discrete_cosine_transform)
    n_dct_filters=40, 
    n_mels=40,
    
    train_pct=80,
    dev_pct=10,
    test_pct=10
)

In [5]:
seed = config["seed"]
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
random.seed(seed)

In [6]:
folder = config["data_folder"]
wanted_words = config["wanted_words"]
unknown_prob = config["unknown_prob"]
train_pct = config["train_pct"]
dev_pct = config["dev_pct"]
test_pct = config["test_pct"]

LABEL_UNKNOWN = "__unknown__"
LABEL_SILENCE = "__silence__"

words = {word: i + 2 for i, word in enumerate(wanted_words)}
words.update({LABEL_SILENCE:0, LABEL_UNKNOWN:1})
sets = [{}, {}, {}]  # [train, dev, test] => {wav_name: label}
unknowns = [0] * 3 # number of wavs for each dataset to take from collected unknown waves according to unknown_prob
bg_noise_files = []
unknown_files = []

In [7]:
words

{'__silence__': 0,
 '__unknown__': 1,
 'down': 5,
 'go': 11,
 'left': 6,
 'no': 3,
 'off': 9,
 'on': 8,
 'right': 7,
 'stop': 10,
 'up': 4,
 'yes': 2}

### prep datasets

In [9]:
max_no_wavs = 2**27 - 1

for folder_name in os.listdir(folder):
    print("Process " + folder_name)
    path_name = os.path.join(folder, folder_name)
    is_bg_noise = False
    
    # Skip files
    if os.path.isfile(path_name):
        continue
    
    # Assign the label based on the current folder name
    if folder_name in words:
        label = words[folder_name]
    elif folder_name == "_background_noise_":
        is_bg_noise = True
    else:
        label = words[LABEL_UNKNOWN]

    for filename in os.listdir(path_name):
        wav_name = os.path.join(path_name, filename)
        
        # Just record background noises files and continue
        if is_bg_noise and os.path.isfile(wav_name):
            bg_noise_files.append(wav_name)
            continue
        # collect wav names when label is unknown
        elif label == words[LABEL_UNKNOWN]:
            unknown_files.append(wav_name)
            continue
        
        # Fill sets with {wave_name : label}
        if config["group_speakers_by_id"]:
            hashname = re.sub(r"_nohash_.*$", "", filename)
            bucket = int(hashlib.sha1(hashname.encode()).hexdigest(), 16)
            bucket = (bucket % (max_no_wavs + 1)) * (100. / max_no_wavs)
            if bucket < dev_pct:
                tag = DatasetType.DEV
            elif bucket < test_pct + dev_pct:
                tag = DatasetType.TEST
            else:
                tag = DatasetType.TRAIN
            sets[tag.value][wav_name] = label
        else:
            raise NotImplementedError("Speakers should be grouped for now :/")
            
# Mix in unknown wavs in datasets
# 1: compute how many files should be mixed in each dataset according to unknown_prob
for tag in range(len(sets)):
    unknowns[tag] = int(unknown_prob * len(sets[tag]))

# 2: shuffle unknown files
random.shuffle(unknown_files)

# 3: update datasets with unknown_files[a:b], where a:b is the interval
a = 0
for tag, dataset in enumerate(sets):
    b = a + unknowns[tag]
    unk_dict = {u: words[LABEL_UNKNOWN] for u in unknown_files[a:b]}
    dataset.update(unk_dict)
    a = b

train_cfg = ChainMap(dict(bg_noise_files=bg_noise_files), config)
test_cfg = ChainMap(dict(noise_prob=0), config)
datasets = (
    SpeechDataset(sets[0], DatasetType.TRAIN, train_cfg), 
    SpeechDataset(sets[1], DatasetType.DEV, test_cfg),
    SpeechDataset(sets[2], DatasetType.TEST, test_cfg)
)

print("Speech dataset created")

Process marvin
Process wow
Process down
Process dog
Process cat
Process on
Process yes
Process left
Process no
Process five
Process house
Process eight
Process _background_noise_
Process nine
Process stop
Process two
Process happy
Process seven
Process zero
Process .DS_Store
Process bird
Process sheila
Process go
Process six
Process off
Process bed
Process three
Process tree
Process up
Process four
Process one
Process right
Speech dataset created


In [10]:
train_set, dev_set, test_set = datasets

## Training

In [11]:
# https://arxiv.org/pdf/1710.10361.pdf
class SpeechResModel(SerializableModule):
    def __init__(self, config):
        super().__init__()
        n_labels = config["n_labels"]
        n_maps = config["n_feature_maps"]
        self.conv0 = nn.Conv2d(1, n_maps, (3, 3), padding=(1, 1), bias=False)
        if "res_pool" in config:
            self.pool = nn.AvgPool2d(config["res_pool"])

        self.n_layers = n_layers = config["n_layers"]
        dilation = config["use_dilation"]
        if dilation:
            self.convs = [nn.Conv2d(n_maps, n_maps, (3, 3), padding=int(2**(i // 3)), dilation=int(2**(i // 3)), 
                bias=False) for i in range(n_layers)]
        else:
            self.convs = [nn.Conv2d(n_maps, n_maps, (3, 3), padding=1, dilation=1, 
                bias=False) for _ in range(n_layers)]
        for i, conv in enumerate(self.convs):
            self.add_module("bn{}".format(i + 1), nn.BatchNorm2d(n_maps, affine=False))
            self.add_module("conv{}".format(i + 1), conv)
        self.output = nn.Linear(n_maps, n_labels)

    def forward(self, x):
        x = x.unsqueeze(1)
        for i in range(self.n_layers + 1):
            y = F.relu(getattr(self, "conv{}".format(i))(x))
            if i == 0:
                if hasattr(self, "pool"):
                    y = self.pool(y)
                old_x = y
            if i > 0 and i % 2 == 0:
                x = y + old_x
                old_x = x
            else:
                x = y
            if i > 0:
                x = getattr(self, "bn{}".format(i))(x)
        x = x.view(x.size(0), x.size(1), -1) # shape: (batch, feats, o3)
        x = torch.mean(x, 2)
        return self.output(x)

In [12]:
res15 = dict(n_labels=12, use_dilation=True, n_layers=13, n_feature_maps=45)
model = SpeechResModel(res15)

torch.cuda.set_device(config["gpu_no"])
model.cuda()

SpeechResModel(
  (conv0): Conv2d (1, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv1): Conv2d (45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv2): Conv2d (45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv3): Conv2d (45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn4): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv4): Conv2d (45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
  (bn5): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv5): Conv2d (45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2), bias=False)
  (bn6): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv6): Conv2d (45, 45, 

In [22]:
def print_eval(name, scores, labels, loss, end="\n"):
    batch_size = labels.size(0)
    accuracy = (torch.max(scores, 1)[1].view(batch_size).data == labels.data).sum() / batch_size
    loss = loss.cpu().data.numpy()[0]
    print("{} accuracy: {:>5}, loss: {:<25}".format(name, accuracy, loss), end=end)
    return accuracy

def evaluate(config, model, test_loader):
    torch.cuda.set_device(config["gpu_no"])
    model.eval()
    criterion = nn.CrossEntropyLoss()
    results = []
    total = 0
    for model_in, labels in test_loader:
        model_in = Variable(model_in, requires_grad=False)
        
        model_in = model_in.cuda()
        labels = labels.cuda()
        
        scores = model(model_in)
        labels = Variable(labels, requires_grad=False)
        loss = criterion(scores, labels)
        results.append(print_eval("test", scores, labels, loss) * model_in.size(0))
        total += model_in.size(0)
    print("final test accuracy: {}".format(sum(results) / total))

In [23]:
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=config["lr"][0], 
    nesterov=config["use_nesterov"], 
    weight_decay=config["weight_decay"], 
    momentum=config["momentum"]
)
schedule_steps = config["schedule"]
schedule_steps.append(np.inf)
sched_idx = 0
criterion = nn.CrossEntropyLoss()
max_acc = 0

train_loader = data.DataLoader(train_set, batch_size=config["batch_size"], shuffle=True, drop_last=True)
dev_loader = data.DataLoader(dev_set, batch_size=min(len(dev_set), 16), shuffle=True)
test_loader = data.DataLoader(test_set, batch_size=min(len(test_set), 16), shuffle=True)
step_no = 0

for epoch_idx in range(config["n_epochs"]):
    for batch_idx, (model_in, labels) in enumerate(train_loader):
        model.train() # sets the model in training mode
        optimizer.zero_grad() # Zero gradients
        
        model_in = model_in.cuda()
        labels = labels.cuda()
        
        model_in = Variable(model_in, requires_grad=False)
        scores = model(model_in)
        
        labels = Variable(labels, requires_grad=False)
        loss = criterion(scores, labels)
        loss.backward() # perform a backward pass
        
        optimizer.step() # and update the weights
        step_no += 1
        if step_no > schedule_steps[sched_idx]:
            sched_idx += 1
            print("changing learning rate to {}".format(config["lr"][sched_idx]))
            optimizer = torch.optim.SGD(model.parameters(), lr=config["lr"][sched_idx],
                nesterov=config["use_nesterov"], momentum=config["momentum"], weight_decay=config["weight_decay"])
        print_eval("train step #{}".format(step_no), scores, labels, loss)

    if epoch_idx % config["dev_every"] == config["dev_every"] - 1:
        model.eval() # sets the model in evaluation mode
        accs = []
        for model_in, labels in dev_loader:
            model_in = Variable(model_in, requires_grad=False)
            model_in = model_in.cuda()
            labels = labels.cuda()
            scores = model(model_in)
            labels = Variable(labels, requires_grad=False)
            loss = criterion(scores, labels)
            loss_numeric = loss.cpu().data.numpy()[0]
            accs.append(print_eval("dev", scores, labels, loss))
        avg_acc = np.mean(accs)
        print("final dev accuracy: {}".format(avg_acc))
        if avg_acc > max_acc:
            print("saving best model...")
            max_acc = avg_acc
            model.save(config["output_file"])
evaluate(config, model, test_loader)

train step #1 accuracy: 0.078125, loss: 2.478188991546631        
train step #2 accuracy: 0.09375, loss: 2.5016794204711914       
train step #3 accuracy: 0.03125, loss: 2.4945156574249268       
train step #4 accuracy: 0.03125, loss: 2.4963042736053467       
train step #5 accuracy: 0.15625, loss: 2.475476026535034        
train step #6 accuracy: 0.09375, loss: 2.4538135528564453       
train step #7 accuracy: 0.078125, loss: 2.4516944885253906       
train step #8 accuracy: 0.125, loss: 2.4377880096435547       
train step #9 accuracy: 0.125, loss: 2.455156087875366        
train step #10 accuracy: 0.234375, loss: 2.4315528869628906       
train step #11 accuracy: 0.203125, loss: 2.424142599105835        
train step #12 accuracy: 0.078125, loss: 2.423679828643799        
train step #13 accuracy: 0.125, loss: 2.3908770084381104       
train step #14 accuracy: 0.109375, loss: 2.348961114883423        
train step #15 accuracy: 0.171875, loss: 2.3722429275512695       
train step #16 acc

train step #125 accuracy: 0.546875, loss: 1.5074219703674316       
train step #126 accuracy: 0.53125, loss: 1.3601495027542114       
train step #127 accuracy: 0.40625, loss: 1.4737846851348877       
train step #128 accuracy: 0.515625, loss: 1.2262933254241943       
train step #129 accuracy: 0.46875, loss: 1.4745912551879883       
train step #130 accuracy: 0.4375, loss: 1.5535123348236084       
train step #131 accuracy: 0.515625, loss: 1.4332613945007324       
train step #132 accuracy: 0.609375, loss: 1.1841676235198975       
train step #133 accuracy: 0.59375, loss: 1.2551127672195435       
train step #134 accuracy:   0.5, loss: 1.3790018558502197       
train step #135 accuracy: 0.515625, loss: 1.314753770828247        
train step #136 accuracy: 0.609375, loss: 1.3310905694961548       
train step #137 accuracy: 0.546875, loss: 1.3479448556900024       
train step #138 accuracy: 0.515625, loss: 1.3389321565628052       
train step #139 accuracy: 0.671875, loss: 1.1904875040054

train step #247 accuracy: 0.71875, loss: 0.9378808736801147       
train step #248 accuracy: 0.6875, loss: 1.0291564464569092       
train step #249 accuracy: 0.71875, loss: 0.812649130821228        
train step #250 accuracy: 0.6875, loss: 1.0779352188110352       
train step #251 accuracy: 0.703125, loss: 0.8310824632644653       
train step #252 accuracy: 0.609375, loss: 1.1507413387298584       
train step #253 accuracy: 0.578125, loss: 1.1010152101516724       
train step #254 accuracy: 0.703125, loss: 0.8615227341651917       
train step #255 accuracy: 0.765625, loss: 0.796261727809906        
train step #256 accuracy: 0.65625, loss: 1.0495136976242065       
train step #257 accuracy: 0.734375, loss: 0.8052343726158142       
train step #258 accuracy: 0.828125, loss: 0.6626462936401367       
train step #259 accuracy: 0.671875, loss: 1.0098766088485718       
train step #260 accuracy: 0.71875, loss: 1.0065826177597046       
train step #261 accuracy: 0.6875, loss: 0.97114396095275

dev accuracy: 0.9375, loss: 0.5912676453590393       
dev accuracy:  0.75, loss: 0.9014200568199158       
dev accuracy: 0.8125, loss: 0.6786653995513916       
dev accuracy: 0.6875, loss: 0.992845892906189        
dev accuracy: 0.9375, loss: 0.5874707698822021       
dev accuracy: 0.875, loss: 0.8343795537948608       
dev accuracy: 0.625, loss: 1.5827196836471558       
dev accuracy: 0.6875, loss: 1.1307759284973145       
dev accuracy:  0.75, loss: 0.7607854604721069       
dev accuracy:   0.5, loss: 1.5315074920654297       
dev accuracy: 0.8125, loss: 0.7784029245376587       
dev accuracy: 0.9375, loss: 0.4699128270149231       
dev accuracy: 0.875, loss: 0.5683741569519043       
dev accuracy: 0.8125, loss: 0.8013444542884827       
dev accuracy: 0.625, loss: 1.3412015438079834       
dev accuracy:  0.75, loss: 0.9572815299034119       
dev accuracy: 0.875, loss: 0.6469597816467285       
dev accuracy: 0.875, loss: 0.6679229736328125       
dev accuracy: 0.625, loss: 0.977177739

dev accuracy: 0.875, loss: 0.8064447641372681       
dev accuracy:  0.75, loss: 1.0199278593063354       
dev accuracy:  0.75, loss: 0.7074865102767944       
dev accuracy: 0.8125, loss: 0.5280083417892456       
dev accuracy:  0.75, loss: 0.9609752297401428       
dev accuracy: 0.6875, loss: 1.2425576448440552       
dev accuracy: 0.6875, loss: 1.175414800643921        
dev accuracy: 0.8125, loss: 0.7339799404144287       
dev accuracy: 0.625, loss: 1.2195992469787598       
dev accuracy:  0.75, loss: 0.915190577507019        
dev accuracy:  0.75, loss: 0.576938271522522        
dev accuracy: 0.6875, loss: 1.3360960483551025       
dev accuracy: 0.6666666666666666, loss: 0.45454859733581543      
final dev accuracy: 0.7463487972508592
saving best model...
train step #348 accuracy: 0.671875, loss: 0.8103102445602417       
train step #349 accuracy: 0.796875, loss: 0.6990839838981628       
train step #350 accuracy: 0.796875, loss: 0.6088122129440308       
train step #351 accuracy: 0.7

...

train step #8763 accuracy: 0.9375, loss: 0.2011992186307907       
train step #8764 accuracy: 0.96875, loss: 0.11307436972856522      
train step #8765 accuracy: 0.984375, loss: 0.06270374357700348      
train step #8766 accuracy: 0.984375, loss: 0.10123256593942642      
train step #8767 accuracy: 0.984375, loss: 0.0925520807504654       
train step #8768 accuracy: 0.953125, loss: 0.24216672778129578      
train step #8769 accuracy: 0.953125, loss: 0.16076302528381348      
train step #8770 accuracy: 0.921875, loss: 0.2862793803215027       
train step #8771 accuracy: 0.96875, loss: 0.13643169403076172      
train step #8772 accuracy: 0.96875, loss: 0.20923790335655212      
train step #8773 accuracy: 0.984375, loss: 0.1192551851272583       
train step #8774 accuracy: 0.984375, loss: 0.04288510978221893      
train step #8775 accuracy:   1.0, loss: 0.06836123764514923      
train step #8776 accuracy: 0.984375, loss: 0.07463914901018143      
train step #8777 accuracy:   1.0, loss: 0.

train step #8884 accuracy: 0.96875, loss: 0.0991571694612503       
train step #8885 accuracy: 0.96875, loss: 0.1680261790752411       
train step #8886 accuracy: 0.96875, loss: 0.1706988662481308       
train step #8887 accuracy: 0.953125, loss: 0.13455641269683838      
train step #8888 accuracy: 0.96875, loss: 0.11194086074829102      
train step #8889 accuracy:   1.0, loss: 0.029042057693004608     
train step #8890 accuracy: 0.984375, loss: 0.08233031630516052      
train step #8891 accuracy: 0.953125, loss: 0.23741625249385834      
train step #8892 accuracy: 0.984375, loss: 0.06875140964984894      
train step #8893 accuracy: 0.96875, loss: 0.06994032114744186      
train step #8894 accuracy: 0.96875, loss: 0.10164930671453476      
train step #8895 accuracy: 0.921875, loss: 0.22693946957588196      
train step #8896 accuracy: 0.953125, loss: 0.2036258578300476       
train step #8897 accuracy: 0.96875, loss: 0.11145221441984177      
train step #8898 accuracy:   1.0, loss: 0.02

train step #9005 accuracy: 0.953125, loss: 0.22378776967525482      
train step #9006 accuracy: 0.953125, loss: 0.1931251883506775       
train step #9007 accuracy: 0.96875, loss: 0.19103071093559265      
train step #9008 accuracy: 0.984375, loss: 0.03447716683149338      
train step #9009 accuracy: 0.96875, loss: 0.1613484025001526       
train step #9010 accuracy: 0.96875, loss: 0.11439330875873566      
train step #9011 accuracy: 0.96875, loss: 0.10079407691955566      
train step #9012 accuracy: 0.96875, loss: 0.08840620517730713      
train step #9013 accuracy: 0.953125, loss: 0.12818461656570435      
train step #9014 accuracy: 0.96875, loss: 0.05477406829595566      
train step #9015 accuracy: 0.96875, loss: 0.12988345324993134      
train step #9016 accuracy:   1.0, loss: 0.020442456007003784     
train step #9017 accuracy: 0.953125, loss: 0.11765875667333603      
train step #9018 accuracy: 0.96875, loss: 0.15828309953212738      
train step #9019 accuracy: 0.96875, loss: 0.1

dev accuracy: 0.9375, loss: 0.23195970058441162      
dev accuracy: 0.9375, loss: 0.34474772214889526      
dev accuracy:   1.0, loss: 0.04554641246795654      
dev accuracy: 0.875, loss: 0.3456769287586212       
dev accuracy:   1.0, loss: 0.004736065864562988     
dev accuracy:   1.0, loss: 0.024448364973068237     
dev accuracy: 0.9375, loss: 0.4244897663593292       
dev accuracy: 0.9375, loss: 0.42405635118484497      
dev accuracy: 0.875, loss: 0.19665458798408508      
dev accuracy:   1.0, loss: 0.1442660242319107       
dev accuracy: 0.9375, loss: 0.2419506013393402       
dev accuracy: 0.9375, loss: 0.18000692129135132      
dev accuracy: 0.8125, loss: 0.46109670400619507      
dev accuracy: 0.9375, loss: 0.10697042942047119      
dev accuracy:   1.0, loss: 0.01246267557144165      
dev accuracy:   1.0, loss: 0.024306774139404297     
dev accuracy: 0.9375, loss: 0.10578089952468872      
dev accuracy: 0.875, loss: 0.335683673620224        
dev accuracy: 0.9375, loss: 0.2108314

test accuracy:   1.0, loss: 0.013709187507629395     
test accuracy:   1.0, loss: 0.0327606201171875       
test accuracy: 0.9375, loss: 0.20119708776474         
test accuracy: 0.875, loss: 0.7215802669525146       
test accuracy:   1.0, loss: 0.0037711262702941895    
test accuracy:   1.0, loss: 0.017562448978424072     
test accuracy: 0.9375, loss: 0.20317262411117554      
test accuracy:   1.0, loss: 0.0036230087280273438    
test accuracy:   1.0, loss: 0.006030678749084473     
test accuracy:   1.0, loss: 0.009665250778198242     
test accuracy:   1.0, loss: 0.026600241661071777     
test accuracy: 0.9375, loss: 0.12577015161514282      
test accuracy:   1.0, loss: 0.003833949565887451     
test accuracy:   1.0, loss: 0.039680227637290955     
test accuracy:   1.0, loss: 0.011337071657180786     
test accuracy:   1.0, loss: 0.09160539507865906      
test accuracy:   1.0, loss: 0.024337172508239746     
test accuracy:   1.0, loss: 0.09897445142269135      
test accuracy: 0.9375, lo

In [53]:
for model_in, labels in test_loader:
    model_in = Variable(model_in, requires_grad=False).cuda()
    labels = labels.cuda()
    scores = model(model_in)
    break

In [84]:
print(labels[0])
print(int(scores[0].max(0)[1]))

3
3


---

### Make submission

In [48]:
label2word = {v: k.strip("_") for k, v in words.items()}

In [13]:
model.load(config["output_file"])

In [32]:
eval_folder = "./../raw_data/test/audio/"
eval_wavs = {
    os.path.join(eval_folder, file_path): os.path.join(eval_folder, file_path) # TODO: this is hack, be careful
        for file_path in os.listdir("./../raw_data/test/audio/")
            if file_path.endswith(".wav")
}

In [33]:
len(eval_wavs)

158538

In [34]:
eval_set = SpeechDataset(eval_wavs, DatasetType.TEST, test_cfg)
eval_loader = data.DataLoader(eval_set, batch_size=16)

In [50]:
with open("../submission.txt", "w") as sub:
    sub.write("fname,label\n")
    for model_in, wav_paths in eval_loader:
        scores = model(Variable(model_in, requires_grad=False).cuda())
        for i, scorei in enumerate(scores):
            sub.write(
                wav_paths[i].rsplit("/")[-1] + "," + label2word[int(scorei.max(0)[1])] + "\n"
            )

AttributeError: 'int' object has no attribute 'rsplit'

In [47]:
"no".strip("_")

'no'

In [36]:
print(label2word[int(scores[4].max(0)[1])])

no


In [37]:
play_wav(wav_paths[4])